In [49]:
import pandas as pd
import geopandas as gpd
import numpy as np

zcta = pd.read_csv('Maryland_Census_Data_-_ZIP_Code_Tabulation_Areas__ZCTAs_.csv')
zipcodes = gpd.read_file('zipcode.geojson')
neighs = gpd.read_file('Maryland_Baltimore_City_Neighborhoods.geojson')
mhinc = pd.read_csv('MASTER_MERGED.csv')
covid = pd.read_csv('COVID_Cleaned_Transposed.csv')
pops = pd.read_csv('PopsByZip.csv')

sjoined = gpd.sjoin(neighs, zipcodes, how='inner',op='intersects')

sjoined.to_csv('spatialJoin.csv')
#zipCleaned = pd.read_csv('ZipData_CLEANED.csv')
#zipsList = [21227,21207,21230,21251,21229,21237,21287,21231,21226,21206,21222,21225,21211,21208,\
#            21205,21218,21234,21202,21201,21213,21210,21209,21216,21236,21217,21224,21215,21212,\
#                21223,21214,21239,21228]

print(zcta.columns)

Index(['OBJECTID_1', 'ZCTA5CE10', 'FIRST_STAT', 'FIRST_GEOI', 'FIRST_CLAS',
       'FIRST_MTFC', 'FIRST_FUNC', 'ZCTA5N', 'STATE', 'AREALAND', 'AREAWATR',
       'POP100', 'HU100', 'NHW', 'NHB', 'NHAI', 'NHA', 'NHNH', 'NHO', 'NHT',
       'HISP', 'PNHW', 'PNHB', 'PNHAI', 'PNHA', 'PNHNH', 'PNHO', 'PNHT',
       'PHISP', 'POP65_', 'PCTPOP65_', 'MEDAGE', 'VACNS', 'PVACNS', 'PHOWN',
       'PWOMORT', 'PRENT', 'PLT18SP', 'REPORT_2_P', 'REPORT_9_P'],
      dtype='object')


In [50]:
sjoined = sjoined[['LABEL','zipcode1']]
sjoined = sjoined.rename(columns={'LABEL':'Neigh','zipcode1':'Zip Code'})
mhinc = mhinc[['Neigh','CSA','MHINC','Pop2010']]

merged = pd.merge_ordered(mhinc, sjoined, fill_method='none')
#print(merged[['CSA','Pop2010']])
merged['Zip Code']= merged['Zip Code'].fillna(0)
merged['Zip Code'] = merged['Zip Code'].astype(int)
merged = merged.sort_values(by=['Zip Code'])
print(merged.head())

     Neigh  CSA  MHINC  Pop2010  Zip Code
1802   NaN  NaN    NaN      NaN         0
1655   NaN  NaN    NaN      NaN         0
1656   NaN  NaN    NaN      NaN         0
1657   NaN  NaN    NaN      NaN         0
1658   NaN  NaN    NaN      NaN         0


In [51]:
#incomeMeans = []
#zipsList.sort()

#for zipcode in zipsList: 
#    df1 = merged[merged['Zip Code'] == zipcode]
#    incMean = df1['MHINC'].unique().mean()
#    incomeMeans.append(incMean)

In [52]:
latestCases = pd.DataFrame()
covid = covid.sort_values(by=['Zip Code'])
latestCases['Zip Code'] = covid['Zip Code'].astype(int)
#latestCases['MHINC'] = incomeMeans
#latestCases['Pop2010'] = popSums
latestCases['TotalCases'] = covid.iloc[:,-1].astype(int)
latestCases = pd.merge_ordered(latestCases,pops,fill_method='none')
latestCases = latestCases.drop('Unnamed: 0',axis=1)
print(latestCases)

     Zip Code  TotalCases  Pop2010
0       20010           0      NaN
1       20015           0      NaN
2       20037           0      NaN
3       20074           0      NaN
4       20161           0      NaN
..        ...         ...      ...
511     21917         134      NaN
512     21918         208      NaN
513     21919         143      NaN
514     21921        2875      NaN
515     21930           0      NaN

[516 rows x 3 columns]


In [53]:
zcta = zcta.rename(columns={'ZCTA5N':'Zip Code','POP100':'Pop2010','PCTPOP65_':'PCTPOP65'})
zcta = zcta.iloc[:,7:38]
zcta = zcta.drop(columns={'STATE'})
print(zcta)

     Zip Code   AREALAND  AREAWATR  Pop2010  HU100    NHW    NHB  NHAI   NHA  \
0       20601  115635266    387684    24156   8722   9785  11146   155   880   
1       20602   35830723    352762    24955   9736   8466  13054   116   731   
2       20603   44239637    219356    28967  10317   9625  15025    98  1446   
3       20606    7501011   1248760      431    230    377     45     1     4   
4       20607   54357590    448221     9802   3504   2165   6321    33   560   
..        ...        ...       ...      ...    ...    ...    ...   ...   ...   
463     21918   45741330   6628177     4284   1589   4096     55     6    30   
464     21919  112736603  22553121     3467   2179   3321     43    14     8   
465     21920    1290392         0      257    100    228     17     0     0   
466     21921  244939298  14098236    44471  17232  36707   3953    98   673   
467     21930    1907039    186920       94     58     67     21     0     1   

     NHNH  ...  PHISP  POP65_  PCTPOP65

In [54]:
#zcta = zcta[zcta['Zip Code'].isin(zipsList)]
zcta.reset_index(drop=True,inplace=True)
zcta = zcta.append({'Zip Code':21287,'PCTPOP65':0,'Pop2010':0},ignore_index=True)
zcta = zcta.fillna(0)
print(zcta)

     Zip Code     AREALAND    AREAWATR  Pop2010    HU100      NHW      NHB  \
0     20601.0  115635266.0    387684.0  24156.0   8722.0   9785.0  11146.0   
1     20602.0   35830723.0    352762.0  24955.0   9736.0   8466.0  13054.0   
2     20603.0   44239637.0    219356.0  28967.0  10317.0   9625.0  15025.0   
3     20606.0    7501011.0   1248760.0    431.0    230.0    377.0     45.0   
4     20607.0   54357590.0    448221.0   9802.0   3504.0   2165.0   6321.0   
..        ...          ...         ...      ...      ...      ...      ...   
464   21919.0  112736603.0  22553121.0   3467.0   2179.0   3321.0     43.0   
465   21920.0    1290392.0         0.0    257.0    100.0    228.0     17.0   
466   21921.0  244939298.0  14098236.0  44471.0  17232.0  36707.0   3953.0   
467   21930.0    1907039.0    186920.0     94.0     58.0     67.0     21.0   
468   21287.0          0.0         0.0      0.0      0.0      0.0      0.0   

      NHAI     NHA  NHNH  ...  PHISP  POP65_  PCTPOP65  MEDAGE 

In [55]:

zcta['Zip Code'] = zcta['Zip Code'].astype(int)

print(zcta)

     Zip Code     AREALAND    AREAWATR  Pop2010    HU100      NHW      NHB  \
0       20601  115635266.0    387684.0  24156.0   8722.0   9785.0  11146.0   
1       20602   35830723.0    352762.0  24955.0   9736.0   8466.0  13054.0   
2       20603   44239637.0    219356.0  28967.0  10317.0   9625.0  15025.0   
3       20606    7501011.0   1248760.0    431.0    230.0    377.0     45.0   
4       20607   54357590.0    448221.0   9802.0   3504.0   2165.0   6321.0   
..        ...          ...         ...      ...      ...      ...      ...   
464     21919  112736603.0  22553121.0   3467.0   2179.0   3321.0     43.0   
465     21920    1290392.0         0.0    257.0    100.0    228.0     17.0   
466     21921  244939298.0  14098236.0  44471.0  17232.0  36707.0   3953.0   
467     21930    1907039.0    186920.0     94.0     58.0     67.0     21.0   
468     21287          0.0         0.0      0.0      0.0      0.0      0.0   

      NHAI     NHA  NHNH  ...  PHISP  POP65_  PCTPOP65  MEDAGE 

In [56]:
print(latestCases)

     Zip Code  TotalCases  Pop2010
0       20010           0      NaN
1       20015           0      NaN
2       20037           0      NaN
3       20074           0      NaN
4       20161           0      NaN
..        ...         ...      ...
511     21917         134      NaN
512     21918         208      NaN
513     21919         143      NaN
514     21921        2875      NaN
515     21930           0      NaN

[516 rows x 3 columns]


In [57]:
zipData = pd.merge_ordered(zcta,latestCases,on='Zip Code',fill_method='none')
print(zipData)

     Zip Code     AREALAND    AREAWATR  Pop2010_x    HU100      NHW     NHB  \
0       20010          NaN         NaN        NaN      NaN      NaN     NaN   
1       20015          NaN         NaN        NaN      NaN      NaN     NaN   
2       20037          NaN         NaN        NaN      NaN      NaN     NaN   
3       20074          NaN         NaN        NaN      NaN      NaN     NaN   
4       20161          NaN         NaN        NaN      NaN      NaN     NaN   
..        ...          ...         ...        ...      ...      ...     ...   
531     21918   45741330.0   6628177.0     4284.0   1589.0   4096.0    55.0   
532     21919  112736603.0  22553121.0     3467.0   2179.0   3321.0    43.0   
533     21920    1290392.0         0.0      257.0    100.0    228.0    17.0   
534     21921  244939298.0  14098236.0    44471.0  17232.0  36707.0  3953.0   
535     21930    1907039.0    186920.0       94.0     58.0     67.0    21.0   

     NHAI    NHA  NHNH  ...  PCTPOP65  MEDAGE  VACN

In [58]:
zipData.to_csv('ZipData_CLEANED.csv')